# Parallel Configuration

This Jupyter notebook will walk you through using threads to execute commands that configure your experiment. 

## Configure the Environment

Before running this notebook, you will need to configure you environment using the [Configure Environment](./configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).


## Import the FABLlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Create the Experiment Slice

The following creates a set of 10 nodes with basic compute capabilities on random sites.


In [ ]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice")

    for i in range(10):
        # Add a node
        node = slice.add_node(name=f"Node{i}")

    #Submit the Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the node

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Configure the Nodes in Parallel


In [ ]:
try:
    config_command = "sudo yum install -q -y net-tools"
    
    #Create execute threads
    execute_threads = {}
    for node in slice.get_nodes():
        print(f"Starting config on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
        print(f"stdout: {stdout}")
        #print(f"stderr: {stderr}")

except Exception as e:
    print(f"Exception: {e}")

## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

In [ ]:
try:
    for node in slice.get_nodes():
        stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s` && netstat -i')
        print(stdout)
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")